In [1]:
# @hidden_cell
# The project token is an authorization token that is used to access project resources like data sources, connections, and used by platform APIs.
from project_lib import Project
project = Project(spark.sparkContext, '159ae44d-b4ce-42c6-ab63-0bfe1667f788', 'p-4d1c4a1dae923b39f95cb7add680a8395249a400')
pc = project.project_context

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20210704063841-0001
KERNEL_ID = f7ba1db3-78de-46b3-9f1d-a46479a63d35


In [2]:
from pyspark.sql.functions import from_unixtime, substring, isnan, when, count, col, coalesce, datediff, to_date, udf, mean, avg
from pyspark.sql.types import TimestampType

In [3]:
import ibmos2spark, os
# @hidden_cell

if os.environ.get('RUNTIME_ENV_LOCATION_TYPE') == 'external':
    endpoint_13cf4ecbeb27425d8d6bd235f70ba8aa = 'https://s3.us.cloud-object-storage.appdomain.cloud'
else:
    endpoint_13cf4ecbeb27425d8d6bd235f70ba8aa = 'https://s3.private.us.cloud-object-storage.appdomain.cloud'

credentials = {
    'endpoint': endpoint_13cf4ecbeb27425d8d6bd235f70ba8aa,
    'service_id': 'iam-ServiceId-f4da2d05-a3ae-4415-b470-90a203bd098d',
    'iam_service_endpoint': 'https://iam.cloud.ibm.com/oidc/token',
    'api_key': 'nmau_4WXcuYuyJ98AmVZEBxzCzhcted8H8tBlTZ6jTTs'
}

configuration_name = 'os_13cf4ecbeb27425d8d6bd235f70ba8aa_configs'
cos = ibmos2spark.CloudObjectStorage(sc, credentials, configuration_name, 'bluemix_cos')

from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
df_userTable = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('userTable.csv', 'apqor-donotdelete-pr-mtpbpimwfhmlnw'))
df_userTable.take(5)

[Row(UserID='U133159', Signup Date='2018-04-14 07:01:16.202607+00:00', User Segment='C'),
 Row(UserID='U129368', Signup Date='2017-12-02 09:38:41.584270+00:00', User Segment='B'),
 Row(UserID='U109654', Signup Date='2013-03-19 11:38:55+00:00', User Segment='B'),
 Row(UserID='U108998', Signup Date='2018-01-18 08:29:51.627954+00:00', User Segment='C'),
 Row(UserID='U131393', Signup Date='2018-03-27 08:05:28.806800+00:00', User Segment='B')]

In [4]:
df_visitorLogsData = spark.read\
  .format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
  .option('header', 'true')\
  .load(cos.url('VisitorLogsData.csv', 'apqor-donotdelete-pr-mtpbpimwfhmlnw'))
df_visitorLogsData.take(5)

[Row(webClientID='WI10000050298', VisitDateTime='2018-05-07 04:28:45.970', ProductID='pr100631', UserID=None, Activity=None, Browser='Chrome Mobile', OS='Android', City='Chennai', Country='India'),
 Row(webClientID='WI10000025922', VisitDateTime='2018-05-13 07:26:04.964', ProductID='pr100707', UserID=None, Activity=None, Browser='Chrome', OS='Windows', City=None, Country='Taiwan'),
 Row(webClientID='WI100000204522', VisitDateTime='2018-05-11 11:43:42.832', ProductID='pr100030', UserID=None, Activity='click', Browser='Chrome', OS='windows', City='Gurgaon', Country='India'),
 Row(webClientID='WI10000011974', VisitDateTime='2018-05-13 15:20:23.436', ProductID='Pr100192', UserID=None, Activity='CLICK', Browser='Chrome', OS='Windows', City=' ', Country=' '),
 Row(webClientID='WI100000441953', VisitDateTime='2018-05-08 20:44:25.238', ProductID='Pr100762', UserID=None, Activity='click', Browser='Chrome', OS='mac os x', City='Iselin', Country='United States')]

In [5]:
df_userTable.show(truncate=False)

+-------+--------------------------------+------------+
|UserID |Signup Date                     |User Segment|
+-------+--------------------------------+------------+
|U133159|2018-04-14 07:01:16.202607+00:00|C           |
|U129368|2017-12-02 09:38:41.584270+00:00|B           |
|U109654|2013-03-19 11:38:55+00:00       |B           |
|U108998|2018-01-18 08:29:51.627954+00:00|C           |
|U131393|2018-03-27 08:05:28.806800+00:00|B           |
|U101889|2018-01-24 12:50:00.225811+00:00|B           |
|U113233|2017-03-18 09:27:53.079335+00:00|B           |
|U115728|2017-08-18 11:40:29.685549+00:00|B           |
|U132899|2018-03-14 08:32:32.906136+00:00|B           |
|U107492|2018-04-10 20:42:50.794037+00:00|B           |
|U124573|2014-09-25 07:24:32+00:00       |B           |
|U121993|2018-03-30 05:49:26.933324+00:00|B           |
|U125731|2017-12-22 16:57:31.586742+00:00|B           |
|U102932|2018-02-10 20:30:29.736785+00:00|C           |
|U114104|2018-04-01 03:31:46.712373+00:00|C     

In [6]:
df_userTable.printSchema()

root
 |-- UserID: string (nullable = true)
 |-- Signup Date: string (nullable = true)
 |-- User Segment: string (nullable = true)



In [7]:
df_userTable = df_userTable.withColumn("Signup Date", df_userTable['Signup Date'].cast(TimestampType()))

In [8]:
df_userTable.printSchema()

root
 |-- UserID: string (nullable = true)
 |-- Signup Date: timestamp (nullable = true)
 |-- User Segment: string (nullable = true)



In [9]:
df_userTable.select([count(when(col(c).isNull(), c)).alias(c) for c in df_userTable.columns]).show()

+------+-----------+------------+
|UserID|Signup Date|User Segment|
+------+-----------+------------+
|     0|          0|           0|
+------+-----------+------------+



In [10]:
df_visitorLogsData.show(truncate=False)

+--------------+-----------------------+---------+-------+--------+-------------+--------+-----------+-----------------------+
|webClientID   |VisitDateTime          |ProductID|UserID |Activity|Browser      |OS      |City       |Country                |
+--------------+-----------------------+---------+-------+--------+-------------+--------+-----------+-----------------------+
|WI10000050298 |2018-05-07 04:28:45.970|pr100631 |null   |null    |Chrome Mobile|Android |Chennai    |India                  |
|WI10000025922 |2018-05-13 07:26:04.964|pr100707 |null   |null    |Chrome       |Windows |null       |Taiwan                 |
|WI100000204522|2018-05-11 11:43:42.832|pr100030 |null   |click   |Chrome       |windows |Gurgaon    |India                  |
|WI10000011974 |2018-05-13 15:20:23.436|Pr100192 |null   |CLICK   |Chrome       |Windows |           |                       |
|WI100000441953|2018-05-08 20:44:25.238|Pr100762 |null   |click   |Chrome       |mac os x|Iselin     |United St

In [11]:
df_visitorLogsData.printSchema()

root
 |-- webClientID: string (nullable = true)
 |-- VisitDateTime: string (nullable = true)
 |-- ProductID: string (nullable = true)
 |-- UserID: string (nullable = true)
 |-- Activity: string (nullable = true)
 |-- Browser: string (nullable = true)
 |-- OS: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Country: string (nullable = true)



In [12]:
df_visitorLogsData.select([count(when(col(c).isNull(), c)).alias(c) for c in df_visitorLogsData.columns]).show()

+-----------+-------------+---------+-------+--------+-------+---+-------+-------+
|webClientID|VisitDateTime|ProductID| UserID|Activity|Browser| OS|   City|Country|
+-----------+-------------+---------+-------+--------+-------+---+-------+-------+
|          0|       658915|   527137|5937305|  889446|      0|  0|2165831| 397693|
+-----------+-------------+---------+-------+--------+-------+---+-------+-------+



In [13]:
df_visitorLogsData = (df_visitorLogsData
    .withColumn("VisitDateTime", coalesce(from_unixtime(substring("VisitDateTime", 0, 10), format='yyyy-MM-dd HH:mm:ss.SSS'), df_visitorLogsData['VisitDateTime'].cast(TimestampType()))
    .cast(TimestampType())))
df_visitorLogsData.show(truncate=False)

+--------------+-----------------------+---------+-------+--------+-------------+--------+-----------+-----------------------+
|webClientID   |VisitDateTime          |ProductID|UserID |Activity|Browser      |OS      |City       |Country                |
+--------------+-----------------------+---------+-------+--------+-------------+--------+-----------+-----------------------+
|WI10000050298 |2018-05-07 04:28:45.97 |pr100631 |null   |null    |Chrome Mobile|Android |Chennai    |India                  |
|WI10000025922 |2018-05-13 07:26:04.964|pr100707 |null   |null    |Chrome       |Windows |null       |Taiwan                 |
|WI100000204522|2018-05-11 11:43:42.832|pr100030 |null   |click   |Chrome       |windows |Gurgaon    |India                  |
|WI10000011974 |2018-05-13 15:20:23.436|Pr100192 |null   |CLICK   |Chrome       |Windows |           |                       |
|WI100000441953|2018-05-08 20:44:25.238|Pr100762 |null   |click   |Chrome       |mac os x|Iselin     |United St

In [14]:
df_visitorLogsData.select([count(when(col(c).isNull(), c)).alias(c) for c in df_visitorLogsData.columns]).show()

+-----------+-------------+---------+-------+--------+-------+---+-------+-------+
|webClientID|VisitDateTime|ProductID| UserID|Activity|Browser| OS|   City|Country|
+-----------+-------------+---------+-------+--------+-------+---+-------+-------+
|          0|       658915|   527137|5937305|  889446|      0|  0|2165831| 397693|
+-----------+-------------+---------+-------+--------+-------+---+-------+-------+



In [15]:
df_visitorLogsData.createOrReplaceTempView('table2')

In [16]:
df_2 = spark.sql("SELECT * FROM table2 WHERE VisitDateTime >= '2018-05-7 00:00:00'")
df_2 = df_2.sort("VisitDateTime")
df_2.show(truncate=False)

+---------------+-----------------------+---------+-------+--------+--------------+--------+-------------+-------------------+
|webClientID    |VisitDateTime          |ProductID|UserID |Activity|Browser       |OS      |City         |Country            |
+---------------+-----------------------+---------+-------+--------+--------------+--------+-------------+-------------------+
|WI100000129228 |2018-05-07 00:00:01.419|Pr100083 |U107518|CLICK   |Chrome        |Windows |Jounieh      |Lebanon            |
|WI100000856564 |2018-05-07 00:00:01.981|Pr100020 |null   |pageload|Firefox Mobile|Android |null         |Russia             |
|WI100000452359 |2018-05-07 00:00:04.574|Pr100224 |null   |pageload|Chrome        |Linux   |Visakhapatnam|India Visakhapatnam|
|WI100000964526 |2018-05-07 00:00:07.04 |Pr100709 |U136965|pageload|Chrome Mobile |Android |null         |India              |
|WI1000001090419|2018-05-07 00:00:07.535|pr100431 |null   |pageload|Chrome        |Windows |null         |Unite

In [17]:
df_2.count()

5929085

In [18]:
df_2.select([count(when(col(c).isNull(), c)).alias(c) for c in df_2.columns]).show()

+-----------+-------------+---------+-------+--------+-------+---+-------+-------+
|webClientID|VisitDateTime|ProductID| UserID|Activity|Browser| OS|   City|Country|
+-----------+-------------+---------+-------+--------+-------+---+-------+-------+
|          0|            0|   474397|5343193|  800320|      0|  0|1948722| 357929|
+-----------+-------------+---------+-------+--------+-------+---+-------+-------+



In [19]:
df_2.createOrReplaceTempView('table2')

In [20]:
df_2 = spark.sql("SELECT * FROM table2 WHERE UserID IS NOT NULL")
df_2.createOrReplaceTempView('table2')
df_2.show(truncate=False)

+---------------+-----------------------+---------+-------+--------+-------------+-------+----------+--------------+
|webClientID    |VisitDateTime          |ProductID|UserID |Activity|Browser      |OS     |City      |Country       |
+---------------+-----------------------+---------+-------+--------+-------------+-------+----------+--------------+
|WI100000129228 |2018-05-07 00:00:01.419|Pr100083 |U107518|CLICK   |Chrome       |Windows|Jounieh   |Lebanon       |
|WI100000964526 |2018-05-07 00:00:07.04 |Pr100709 |U136965|pageload|Chrome Mobile|Android|null      |India         |
|WI1000007731   |2018-05-07 00:00:10.434|null     |U136963|null    |Chrome Mobile|Android|null      |null          |
|WI1000007731   |2018-05-07 00:00:11.164|Pr100102 |U136963|CLICK   |Chrome Mobile|Android|null      |null          |
|WI1000007731   |2018-05-07 00:00:12.156|pr100102 |U136963|CLICK   |Chrome Mobile|Android|null      |India         |
|WI1000007731   |2018-05-07 00:00:12.398|null     |U136963|click

In [21]:
df_2.select([count(when(col(c).isNull(), c)).alias(c) for c in df_2.columns]).show()

+-----------+-------------+---------+------+--------+-------+---+------+-------+
|webClientID|VisitDateTime|ProductID|UserID|Activity|Browser| OS|  City|Country|
+-----------+-------------+---------+------+--------+-------+---+------+-------+
|          0|            0|    46663|     0|   78890|      0|  0|212654|  38590|
+-----------+-------------+---------+------+--------+-------+---+------+-------+



In [22]:
df_2.count()

585892

In [23]:
df_2.select('Activity').distinct().collect()

[Row(Activity=None),
 Row(Activity='CLICK'),
 Row(Activity='click'),
 Row(Activity='PAGELOAD'),
 Row(Activity='pageload')]

In [24]:
df_userTable.createOrReplaceTempView('table1')

In [25]:
def os_clean(os):
    if (os != os) | (os is None):
        return None
    if os=="android":
        return "Android"
    elif os=="windows":
        return "Windows"
    elif os=="mac os x":
        return "Mac OS X"
    elif os=="ios":
        return "iOS"
    elif os=="linux":
        return "Linux"
    elif os=="ubuntu":
        return "Ubuntu"
    elif os=="chrome os":
        return "Chrome OS"
    else:
        return os

def pr_clean(pr):
    if (pr != pr) | (pr is None):
        return None
    if pr[0].islower():
        return pr[0].upper() + pr[1:]
    else:
        return pr

def act_clean(act):
    if (act != act) | (act is None):
        return None
    if act[0].islower():
        return act[0].upper()+act[1:]
    else:
        return act[0]+act[1:].lower()

In [26]:
osClean = udf(lambda x: os_clean(x))
prClean = udf(lambda x: pr_clean(x))
actClean = udf(lambda x: act_clean(x))

In [27]:
df_2 = df_2.withColumn('OS', osClean('OS'))
df_2 = df_2.withColumn('ProductID', prClean('ProductID'))
df_2 = df_2.withColumn('Activity', actClean('Activity'))

df_2.createOrReplaceTempView('table2')

In [28]:
User_Vintage_df = spark.sql('SELECT `UserID`, DATEDIFF("2018-05-28 00:00:00", `Signup Date`) AS `User_Vintage` FROM table1 ORDER BY UserID ASC')
User_Vintage_df.show(truncate=False)

+-------+------------+
|UserID |User_Vintage|
+-------+------------+
|U100002|53          |
|U100003|1021        |
|U100004|341         |
|U100005|681         |
|U100006|55          |
|U100007|460         |
|U100008|395         |
|U100009|78          |
|U100012|124         |
|U100013|1687        |
|U100014|155         |
|U100015|659         |
|U100016|114         |
|U100017|45          |
|U100018|44          |
|U100019|317         |
|U100020|163         |
|U100023|437         |
|U100024|85          |
|U100025|129         |
+-------+------------+
only showing top 20 rows



In [29]:
User_Vintage_df.count()

34050

In [30]:
Products_15_df = spark.sql('SELECT UserID, COUNT(DISTINCT ProductID) AS `No_Of_Products_Viewed_15_Days` FROM table2 WHERE VisitDateTime >= "2018-05-13 00:00:00" AND ProductID IS NOT NULL GROUP BY UserID ORDER BY UserID')
Products_15_df.show(truncate=False)

+-------+-----------------------------+
|UserID |No_Of_Products_Viewed_15_Days|
+-------+-----------------------------+
|U100002|2                            |
|U100003|2                            |
|U100004|15                           |
|U100005|3                            |
|U100006|1                            |
|U100008|17                           |
|U100009|10                           |
|U100012|5                            |
|U100013|3                            |
|U100014|4                            |
|U100015|4                            |
|U100016|1                            |
|U100017|1                            |
|U100018|3                            |
|U100019|7                            |
|U100020|2                            |
|U100023|3                            |
|U100024|1                            |
|U100026|2                            |
|U100027|41                           |
+-------+-----------------------------+
only showing top 20 rows



In [31]:
Pageloads_7_df = spark.sql('SELECT UserID, COUNT(Activity) AS `Pageloads_last_7_days` FROM table2 WHERE VisitDateTime >= "2018-05-21 00:00:00" AND Activity = "Pageload" GROUP BY UserID ORDER BY UserID')
Pageloads_7_df.show(truncate=False)

+-------+---------------------+
|UserID |Pageloads_last_7_days|
+-------+---------------------+
|U100003|1                    |
|U100004|1                    |
|U100005|1                    |
|U100006|1                    |
|U100008|21                   |
|U100009|4                    |
|U100012|6                    |
|U100013|5                    |
|U100014|1                    |
|U100015|3                    |
|U100016|1                    |
|U100018|7                    |
|U100019|5                    |
|U100023|6                    |
|U100024|2                    |
|U100026|1                    |
|U100027|15                   |
|U100029|1                    |
|U100030|2                    |
|U100031|8                    |
+-------+---------------------+
only showing top 20 rows



In [32]:
Clicks_7_df = spark.sql('SELECT UserID, COUNT(Activity) AS `Clicks_last_7_days` FROM table2 WHERE VisitDateTime >= "2018-05-21 00:00:00" AND Activity = "Click" GROUP BY UserID ORDER BY UserID')
Clicks_7_df.show(truncate=False)

+-------+------------------+
|UserID |Clicks_last_7_days|
+-------+------------------+
|U100003|2                 |
|U100008|23                |
|U100009|4                 |
|U100012|14                |
|U100013|3                 |
|U100014|2                 |
|U100015|8                 |
|U100018|21                |
|U100019|15                |
|U100024|1                 |
|U100026|2                 |
|U100027|36                |
|U100029|1                 |
|U100030|4                 |
|U100031|61                |
|U100032|205               |
|U100034|37                |
|U100036|11                |
|U100038|11                |
|U100040|3                 |
+-------+------------------+
only showing top 20 rows



In [33]:
Visited_7_df = df_2.withColumn("DateOnly", to_date(col("VisitDateTime")))
Visited_7_df.createOrReplaceTempView('table2_dateOnly')
Visited_7_df = spark.sql('SELECT UserID, COUNT(DISTINCT DateOnly) AS `No_of_days_Visited_7_Days` FROM table2_dateOnly WHERE DateOnly >= "2018-05-21" GROUP BY UserID ORDER BY UserID')
Visited_7_df.show(truncate=False)

+-------+-------------------------+
|UserID |No_of_days_Visited_7_Days|
+-------+-------------------------+
|U100003|1                        |
|U100004|1                        |
|U100005|1                        |
|U100006|1                        |
|U100008|6                        |
|U100009|4                        |
|U100012|2                        |
|U100013|3                        |
|U100014|2                        |
|U100015|1                        |
|U100016|1                        |
|U100017|1                        |
|U100018|5                        |
|U100019|3                        |
|U100023|5                        |
|U100024|1                        |
|U100026|3                        |
|U100027|6                        |
|U100029|1                        |
|U100030|3                        |
+-------+-------------------------+
only showing top 20 rows



In [34]:
Recent_prod_df = spark.sql('SELECT * FROM table2 WHERE Activity = "Pageload" ORDER BY VisitDateTime DESC')
Recent_prod_df.createOrReplaceTempView('table2_pageload_sortedDesc')
Recent_prod_df = spark.sql('SELECT UserID, FIRST(ProductID) AS Recently_Viewed_Product FROM table2_pageload_sortedDesc WHERE ProductID IS NOT NULL GROUP BY UserID ORDER BY UserID')
Recent_prod_df.show(truncate=False)

+-------+-----------------------+
|UserID |Recently_Viewed_Product|
+-------+-----------------------+
|U100002|Pr100258               |
|U100003|Pr100079               |
|U100004|Pr100197               |
|U100005|Pr100234               |
|U100006|Pr101111               |
|U100007|Pr100265               |
|U100008|Pr100089               |
|U100009|Pr101070               |
|U100012|Pr100055               |
|U100013|Pr100177               |
|U100014|Pr100126               |
|U100015|Pr100095               |
|U100016|Pr100172               |
|U100018|Pr100403               |
|U100019|Pr100188               |
|U100020|Pr100017               |
|U100023|Pr100181               |
|U100024|Pr100025               |
|U100025|Pr100240               |
|U100026|Pr100153               |
+-------+-----------------------+
only showing top 20 rows



In [35]:
Most_OS_df = spark.sql('SELECT UserID, OS, COUNT(*) AS Count FROM table2 GROUP BY OS, UserID ORDER BY UserID ASC, Count DESC')
Most_OS_df.createOrReplaceTempView('table2_osCount_sortDesc')
Most_OS_df = spark.sql('SELECT UserID, FIRST(OS) AS Most_Active_OS FROM table2_osCount_sortDesc WHERE OS IS NOT NULL GROUP BY UserID ORDER BY UserID')
Most_OS_df.show(truncate=False)

+-------+--------------+
|UserID |Most_Active_OS|
+-------+--------------+
|U100002|Android       |
|U100003|Windows       |
|U100004|Windows       |
|U100005|Android       |
|U100006|Android       |
|U100007|Windows       |
|U100008|Android       |
|U100009|Android       |
|U100012|Mac OS X      |
|U100013|Mac OS X      |
|U100014|Windows       |
|U100015|Android       |
|U100016|Android       |
|U100017|Windows       |
|U100018|Windows       |
|U100019|Windows       |
|U100020|Mac OS X      |
|U100023|Windows       |
|U100024|Android       |
|U100025|Windows       |
+-------+--------------+
only showing top 20 rows



In [36]:
Most_15_df = spark.sql('SELECT UserID, ProductID, COUNT(*) AS Count FROM table2_pageload_sortedDesc WHERE VisitDateTime >= "2018-05-13 00:00:00" GROUP BY ProductID, UserID ORDER BY UserID ASC, Count DESC')
Most_15_df.createOrReplaceTempView('table2_last15_prodCount_sortDesc')
Most_15_df = spark.sql('SELECT UserID, FIRST(ProductID) AS Most_Viewed_product_15_Days FROM table2_last15_prodCount_sortDesc WHERE ProductID IS NOT NULL GROUP BY UserID ORDER BY UserID')
Most_15_df.show(truncate=False)

+-------+---------------------------+
|UserID |Most_Viewed_product_15_Days|
+-------+---------------------------+
|U100002|Pr100258                   |
|U100003|Pr100079                   |
|U100004|Pr101505                   |
|U100005|Pr100234                   |
|U100006|Pr101111                   |
|U100008|Pr100184                   |
|U100009|Pr100167                   |
|U100012|Pr100055                   |
|U100013|Pr100177                   |
|U100014|Pr100126                   |
|U100015|Pr100044                   |
|U100016|Pr100172                   |
|U100018|Pr100403                   |
|U100019|Pr100040                   |
|U100020|Pr100017                   |
|U100023|Pr100181                   |
|U100024|Pr100025                   |
|U100026|Pr100153                   |
|U100027|Pr102905                   |
|U100029|Pr100301                   |
+-------+---------------------------+
only showing top 20 rows



In [37]:
dfs = [User_Vintage_df, Products_15_df, Pageloads_7_df, Clicks_7_df, Visited_7_df, Recent_prod_df, Most_OS_df, Most_15_df]
for df in dfs:
    print(str(df) + ":", df.count())

DataFrame[UserID: string, User_Vintage: int]: 34050
DataFrame[UserID: string, No_Of_Products_Viewed_15_Days: bigint]: 25441
DataFrame[UserID: string, Pageloads_last_7_days: bigint]: 13814
DataFrame[UserID: string, Clicks_last_7_days: bigint]: 10609
DataFrame[UserID: string, No_of_days_Visited_7_Days: bigint]: 16172
DataFrame[UserID: string, Recently_Viewed_Product: string]: 28250
DataFrame[UserID: string, Most_Active_OS: string]: 33357
DataFrame[UserID: string, Most_Viewed_product_15_Days: string]: 21654


In [38]:
final_df = User_Vintage_df
for i in range(1, len(dfs)):
    final_df = final_df.join(dfs[i], "UserID", "left")   
final_df.show()

+-------+------------+-----------------------------+---------------------+------------------+-------------------------+-----------------------+--------------+---------------------------+
| UserID|User_Vintage|No_Of_Products_Viewed_15_Days|Pageloads_last_7_days|Clicks_last_7_days|No_of_days_Visited_7_Days|Recently_Viewed_Product|Most_Active_OS|Most_Viewed_product_15_Days|
+-------+------------+-----------------------------+---------------------+------------------+-------------------------+-----------------------+--------------+---------------------------+
|U100364|         309|                            1|                 null|              null|                     null|                   null|       Windows|                       null|
|U100499|         474|                           50|                  122|              2247|                        7|               Pr101367|       Windows|                   Pr101042|
|U100608|         453|                            3|             

In [39]:
final_df.count()

34050

In [40]:
final_df.select([count(when(col(c).isNull(), c)).alias(c) for c in final_df.columns]).show()

+------+------------+-----------------------------+---------------------+------------------+-------------------------+-----------------------+--------------+---------------------------+
|UserID|User_Vintage|No_Of_Products_Viewed_15_Days|Pageloads_last_7_days|Clicks_last_7_days|No_of_days_Visited_7_Days|Recently_Viewed_Product|Most_Active_OS|Most_Viewed_product_15_Days|
+------+------------+-----------------------------+---------------------+------------------+-------------------------+-----------------------+--------------+---------------------------+
|     0|           0|                         8609|                20236|             23441|                    17878|                   5800|           693|                      12396|
+------+------------+-----------------------------+---------------------+------------------+-------------------------+-----------------------+--------------+---------------------------+



In [41]:
final_df = final_df.select(["UserID",
                "No_of_days_Visited_7_Days",
                "No_Of_Products_Viewed_15_Days",
                "User_Vintage",
                "Most_Viewed_product_15_Days",
                "Most_Active_OS",
                "Recently_Viewed_Product",
                "Pageloads_last_7_days",
                "Clicks_last_7_days"])

In [42]:
final_df = final_df.sort("UserID")

final_df.show()

+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
| UserID|No_of_days_Visited_7_Days|No_Of_Products_Viewed_15_Days|User_Vintage|Most_Viewed_product_15_Days|Most_Active_OS|Recently_Viewed_Product|Pageloads_last_7_days|Clicks_last_7_days|
+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
|U100002|                     null|                            2|          53|                   Pr100258|       Android|               Pr100258|                 null|              null|
|U100003|                        1|                            2|        1021|                   Pr100079|       Windows|               Pr100079|                    1|                 2|
|U100004|                        1|                           15|

In [44]:
def fillMean(df, columns):
    for column in columns:
        stat = df.select(mean(col(column)).alias('mean')).collect()
        df = df.na.fill(int(stat[0][0]), [column])
    return df

final_df = final_df.na.fill("Product101", ["Recently_Viewed_Product", "Most_Viewed_product_15_Days"]) \
                   .na.fill("Windows", ["Most_Active_OS"]) \
                   .na.fill(0, ["Pageloads_last_7_days", "Clicks_last_7_days", "No_of_days_Visited_7_Days", "No_Of_Products_Viewed_15_Days"])

In [45]:
final_df.show(truncate=False)

+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
|UserID |No_of_days_Visited_7_Days|No_Of_Products_Viewed_15_Days|User_Vintage|Most_Viewed_product_15_Days|Most_Active_OS|Recently_Viewed_Product|Pageloads_last_7_days|Clicks_last_7_days|
+-------+-------------------------+-----------------------------+------------+---------------------------+--------------+-----------------------+---------------------+------------------+
|U100002|0                        |2                            |53          |Pr100258                   |Android       |Pr100258               |0                    |0                 |
|U100003|1                        |2                            |1021        |Pr100079                   |Windows       |Pr100079               |1                    |2                 |
|U100004|1                        |15                           |

In [46]:
#project.save_data(file_name = "solution.csv", data = final_df.toPandas().to_csv(index=False), overwrite=True, set_project_asset = True)